In [33]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures, LabelEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV, cross_val_predict
from sklearn.metrics import mean_squared_error, accuracy_score

In [24]:
train = pd.read_csv(r"C:\Users\tuke-\Desktop\1_GSB544_Computing_and_Machine_Learning\Final\Classification\data\CAH-201803-train.csv")

train = train.dropna()

test = pd.read_csv(r"C:\Users\tuke-\Desktop\1_GSB544_Computing_and_Machine_Learning\Final\Classification\data\CAH-201803-test.csv")

test = test.dropna()


id_num                    int64
Q1                       object
Q2                        int64
political_affiliation    object
Q4                       object
Q5                       object
Q6                       object
Q7                       object
Q8                       object
Q9                       object
Q10                      object
Q11                      object
Q12                      object
Q13                      object
Q14                      object
Q15                       int64
Q16                       int64
Q17                       int64
Q18                      object
dtype: object


In [22]:
ct = ColumnTransformer(
  [
    ("dummify", 
    OneHotEncoder(sparse_output = False, handle_unknown='ignore'),
    make_column_selector(dtype_include=object))
  ],
  remainder = "passthrough"
).set_output(transform = "pandas")

In [29]:
X = train.drop(['id_num','political_affiliation'], axis = 1)

y = train[['political_affiliation']]

print(y)

le =LabelEncoder()

le.fit(y)

list(le.classes_)

y = le.transform(y)

print(y)

    political_affiliation
0             Independent
1             Independent
2                Democrat
3             Independent
4              Republican
..                    ...
164            Republican
165            Republican
166            Republican
167            Republican
168            Republican

[169 rows x 1 columns]
[1 1 0 1 2 2 0 0 1 1 0 2 1 0 2 2 0 0 2 1 1 1 1 1 0 1 0 1 2 1 1 1 2 1 0 1 1
 0 2 2 1 2 1 1 2 2 0 1 0 1 1 2 2 0 2 0 0 2 0 1 1 0 1 1 1 0 1 0 1 2 1 0 2 2
 0 1 0 0 0 0 2 1 2 1 1 2 1 1 0 2 0 2 1 2 0 1 0 2 0 2 2 2 0 0 0 0 1 0 1 2 2
 0 1 1 0 0 0 0 2 0 0 2 1 2 1 2 1 2 1 0 2 0 0 1 0 2 1 2 0 0 0 0 1 1 0 2 1 0
 0 0 2 2 0 2 2 0 0 2 2 0 2 1 2 1 2 2 2 2 2]


C:\Users\tuke-\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tuke-\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


In [34]:
from xgboost import XGBClassifier


X_test, X_train, y_test, y_train = train_test_split(X,y, test_size = 0.25)

pipeline = Pipeline(
  [
   ("preprocessing", ct),
  ("Regression", XGBClassifier())]
)

fitted = pipeline.fit(X_train,y_train)

param_grid = {
    'Regression__n_estimators': [100, 200, 300, 400, 500],
    'Regression__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'Regression__max_depth': [3, 4, 5, 6, 7, 8],
    #'Regression__colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1],
    #'Regression__gamma': [0, 0.1, 0.2, 0.3, 0.4, 0.5]
}


grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy', n_jobs=-1)

grid_search.fit(X, y)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

best_score = -grid_search.best_score_
print(f"Best Accuracy score: {best_score}")

y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print('Test accuracy: %f' % accuracy)

Best parameters: {'Regression__learning_rate': 0.05, 'Regression__max_depth': 3, 'Regression__n_estimators': 200}
Best Accuracy score: -0.5922794117647059
Test RMSE: 0.968254


In [43]:
X = train.drop(['id_num','political_affiliation'], axis = 1)

y = train['political_affiliation']

le = LabelEncoder()

le.fit(y)

list(le.classes_)

y = le.transform(y)

pipeline = Pipeline(
  [
   ("preprocessing", ct),
  ("Regression", XGBClassifier(learning_rate = 0.05, max_depth = 3, n_estimators = 200))]
)

fitted = pipeline.fit(X,y)

y = fitted.predict(test)

original_labels = le.inverse_transform(y)

final_predictions = pd.DataFrame(
    {"id_num": test["id_num"],
    "political_affiliation_predicted": original_labels}
)


print(final_predictions)

     id_num political_affiliation_predicted
0         2                      Republican
1         3                        Democrat
2         4                     Independent
3         6                     Independent
4        11                     Independent
..      ...                             ...
161     327                        Democrat
162     330                     Independent
163     331                      Republican
164     333                        Democrat
165     334                      Republican

[166 rows x 2 columns]


In [40]:
final_predictions.to_csv(r'C:\Users\tuke-\Desktop\final_predictions_political.csv', index=False)

In [44]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = train.drop(['id_num','political_affiliation'], axis = 1)

y = train['political_affiliation']

le = LabelEncoder()

le.fit(y)

list(le.classes_)

y = le.transform(y)

X_test, X_train, y_test, y_train = train_test_split(X, y, test_size = 0.25)

pipeline = Pipeline(
  [
   ("preprocessing", ct),
   ("Regression", svm.SVC())
  ]
)

fitted = pipeline.fit(X_train, y_train)

param_grid = {
    'Regression__C': [0.1, 1, 10, 100, 1000],  
    'Regression__gamma': [1, 0.1, 0.01, 0.001, 0.0001], 
    'Regression__kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

grid_search = GridSearchCV(pipeline, param_grid, cv=10, scoring='accuracy', n_jobs=-1)

grid_search.fit(X, y)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")

best_score = -grid_search.best_score_
print(f"Best Accuracy score: {best_score}")

y_pred = grid_search.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print('Test accuracy: %f' % accuracy)


KeyboardInterrupt: 

In [ ]:
X = train.drop(['id_num','political_affiliation'], axis = 1)

y = train['political_affiliation']

le = LabelEncoder()

le.fit(y)

list(le.classes_)

y = le.transform(y)

pipeline = Pipeline(
  [
   ("preprocessing", ct),
  ("Regression", XGBClassifier(learning_rate = 0.05, max_depth = 3, n_estimators = 200))]
)

fitted = pipeline.fit(X,y)

y = fitted.predict(test)

original_labels = le.inverse_transform(y)

final_predictions = pd.DataFrame(
    {"id_num": test["id_num"],
    "political_affiliation_predicted": original_labels}
)


print(final_predictions)

In [ ]:
final_predictions.to_csv(r'C:\Users\tuke-\Desktop\final_predictions_political.csv', index=False)